<a href="https://colab.research.google.com/github/GaryPython/Data-science-marathon/blob/main/Titanic/6_%E5%BE%9E%E8%B3%87%E6%96%99%E4%B8%AD%E7%94%A2%E7%94%9F%E7%89%B9%E5%BE%B5_%E4%BD%9C%E6%A5%ADD39.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 今天的作業
在鐵達尼資料集中，今天我們專注觀察變數之間的相關性，以Titanic_train.csv 中，首先將有遺失值的數值刪除，我們取 Titanic_train.csv 的年齡資料，試著將課堂中所學的方法應用上去。

* Q1: 產生一個新的變數(Age_above65_) Age>=65為 'Y'，其餘為'N'。  
* Q2: 添加女性和男性，產生一個新的變數(Age_above65_female)，女性或Age>=65為'Y'，其餘為'N'。  
*  Q3: 透過昨天課程的內容，驗證產生的兩個新變數，哪一個和目標變數(Survived_cate)的相關性較高?

In [ ]:
!pip3 install pingouin
!pip3 install researchpy


     |████████████████████████████████| 204kB 11.0MB/s 
  Created wheel for pingouin: filename=pingouin-0.3.10-cp36-none-any.whl size=203397 sha256=3c54b1ef32036b2e2c414f6a5d77348f30f8293fcd0423960fd9f1bb9264ce6e
  Stored in directory: /root/.cache/pip/wheels/e6/f4/06/b5046209e6c0d96b4dea243c45e926b5c0dd88e23b01c45f92
  Created wheel for outdated: filename=outdated-0.2.0-cp36-none-any.whl size=4961 sha256=88fadd71d5e36d202d677ede45ea465c39a102c2fb9ef654b868a4e58c0e4ec3
  Stored in directory: /root/.cache/pip/wheels/fd/7c/ef/814f514d31197310872b5abf353feb8fef9d67ee658e1e7e39
  Created wheel for littleutils: filename=littleutils-0.2.2-cp36-none-any.whl size=7051 sha256=874b4ef3b407106a6ac1ad0c8683c6325c3779f8b56bc1018fdfadf6ca372c48
  Stored in directory: /root/.cache/pip/wheels/53/16/9f/ac67d15c40243754fd73f620e1b9b6dedc20492ecc19a2bae1
Successfully built pingouin outdated littleutils


In [ ]:
# library
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import math
import statistics
import seaborn as sns
from IPython.display import display
import sklearn
print(sklearn.__version__)
#如果只有 0.19 記得要更新至 最新版本
%matplotlib inline


#顯示所有行
pd.set_option('display.max_columns', None)
#顯示所有列
pd.set_option('display.max_rows', None)
#顯示行設定
pd.set_option('max_colwidth',100)

import pingouin as pg
import researchpy   
 

0.22.2.post1


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## 讀入資料

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Titanic_test.csv to Titanic_test.csv
Saving Titanic_train.csv to Titanic_train.csv


In [ ]:
df_train = pd.read_csv("Titanic_train.csv")
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [ ]:
#1.產稱一個新的變數 Survived_cate ，資料型態傳換成類別型態      
#2.把題目中的 Survived 用 Survived_cate 來做分析    
df_train['Survived_cate']=df_train['Survived']
df_train['Survived_cate']=df_train['Survived_cate'].astype('object')
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PassengerId    891 non-null    int64  
 1   Survived       891 non-null    int64  
 2   Pclass         891 non-null    int64  
 3   Name           891 non-null    object 
 4   Sex            891 non-null    object 
 5   Age            714 non-null    float64
 6   SibSp          891 non-null    int64  
 7   Parch          891 non-null    int64  
 8   Ticket         891 non-null    object 
 9   Fare           891 non-null    float64
 10  Cabin          204 non-null    object 
 11  Embarked       889 non-null    object 
 12  Survived_cate  891 non-null    object 
dtypes: float64(2), int64(5), object(6)
memory usage: 90.6+ KB
None


In [ ]:
#我們先把遺失值刪除
## 取出資料後，把遺失值刪除
complete_data=df_train[['Age','Survived_cate','Sex']].dropna()

complete_data.head()

,Age,Survived_cate,Sex
0,22.0,0,male
1,38.0,1,female
2,26.0,1,female
3,35.0,1,female
4,35.0,0,male


### Q1: 產生一個新的變數(Age_above65_) Age>=65為 'Y'，其餘為'N'。


In [ ]:
def age_above(x): 
    if(x>=65):
        return('Y')
    else: 
        return('N')


complete_data["Age_above65"] = complete_data["Age"].apply(age_above) 
complete_data.head()


,Age,Survived_cate,Sex,Age_above65
0,22.0,0,male,N
1,38.0,1,female,N
2,26.0,1,female,N
3,35.0,1,female,N
4,35.0,0,male,N


### Q2: 添加女性和男性，產生一個新的變數(Age_above65_female)，女性或Age>=65為'Y'，其餘為'N'。
* 暗示: 觀看下面影片找答案，https://www.youtube.com/watch?v=X2d-wUt5azY

In [ ]:

def age_above_female(row): 
  if (row.Sex=='female'):
        return('Y')
  else: 
    if(row.Age >= 65):
        return('Y')
    else: 
        return('N')



complete_data["Age_above65_female"] = complete_data[['Age','Sex']].apply(age_above_female,axis=1) 
complete_data.head()


,Age,Survived_cate,Sex,Age_above65,Age_above65_female
0,22.0,0,male,N,N
1,38.0,1,female,N,Y
2,26.0,1,female,N,Y
3,35.0,1,female,N,Y
4,35.0,0,male,N,N


### Q3: 透過昨天課程的內容，驗證產生的兩個新變數，哪一個和目標變數(Survived_cate)的相關性較高?
* 提示:    
  首先觀察一下這些變數的資料型態後，再來想要以哪一種判斷倆倆的相關性。     

In [ ]:
#Age_above65
contTable = pd.crosstab(complete_data['Age_above65'], complete_data['Survived_cate'])
print('交叉列連表:\n\n',contTable)
df = min(contTable.shape[0], contTable.shape[1]) - 1
print('\n自由度：',df)
crosstab, res = researchpy.crosstab(complete_data['Survived_cate'], complete_data['Age_above65'], test='chi-square')
print('\nCramer’s V 係數:\n\n',res)
print('\n卡方檢定:\n',crosstab)

print("Cramer's value is",res.loc[2,'results'])
def judgment_CramerV(df,V):
    if df == 1:
        if V < 0.10:
            qual = 'negligible'
        elif V < 0.30:
            qual = 'small'
        elif V < 0.50:
            qual = 'medium'
        else:
            qual = 'large'
    elif df == 2:
        if V < 0.07:
            qual = 'negligible'
        elif V < 0.21:
            qual = 'small'
        elif V < 0.35:
            qual = 'medium'
        else:
            qual = 'large'
    elif df == 3:
        if V < 0.06:
            qual = 'negligible'
        elif V < 0.17:
            qual = 'small'
        elif V < 0.29:
            qual = 'medium'
        else:
            qual = 'large'
    elif df == 4:
        if V < 0.05:
            qual = 'negligible'
        elif V < 0.15:
            qual = 'small'
        elif V < 0.25:
            qual = 'medium'
        else:
            qual = 'large'
    else:
        if V < 0.05:
            qual = 'negligible'
        elif V < 0.13:
            qual = 'small'
        elif V < 0.22:
            qual = 'medium'
        else:
            qual = 'large'
    return(qual)

judgment_CramerV(df,res.loc[2,'results'])

交叉列連表:

 Survived_cate    0    1
Age_above65            
N              414  289
Y               10    1

自由度： 1

Cramer’s V 係數:

                 Chi-square test  results
0  Pearson Chi-square ( 1.0) =    4.6035
1                    p-value =    0.0319
2               Cramer's phi =    0.0803

卡方檢定:
               Age_above65         
Age_above65             N   Y  All
Survived_cate                     
0                     414  10  424
1                     289   1  290
All                   703  11  714
Cramer's value is 0.0803


'negligible'

In [ ]:
#Age_above65_female
contTable_female = pd.crosstab(complete_data['Age_above65_female'], complete_data['Survived_cate'])
print('交叉列連表:\n\n',contTable_female)
df_female = min(contTable.shape[0], contTable.shape[1]) - 1
print('\n自由度：',df_female)

crosstab_female, res_female = researchpy.crosstab(complete_data['Survived_cate'], complete_data['Age_above65_female'], test='chi-square')
print('\nCramer’s V 係數:\n\n',res_female)
print('\n卡方檢定:\n',crosstab_female)
print("Cramer's value is",res.loc[2,'results'])


judgment_CramerV(df,res.loc[2,'results'])


交叉列連表:

 Survived_cate         0    1
Age_above65_female          
N                   350   92
Y                    74  198

自由度： 1

Cramer’s V 係數:

                 Chi-square test   results
0  Pearson Chi-square ( 1.0) =   188.6219
1                    p-value =     0.0000
2               Cramer's phi =     0.5140

卡方檢定:
                    Age_above65_female          
Age_above65_female                  N    Y  All
Survived_cate                                  
0                                 350   74  424
1                                  92  198  290
All                               442  272  714
Cramer's value is 0.514


'large'